In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
# NOTE: the next import is only valid 
# for scikit-learn version <= 0.17
# if you are using scikit-learn >= 0.18 then use this:
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
import os
%matplotlib inline 
from skimage.feature import hog


In [2]:
# play with these values to see how your classifier
# performs under different binning scenarios
color_space = 'HSV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 6  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
no_hist_bins = 48    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
vis = True

In [3]:
basedir = 'vehicles/'
# different folders respresent different 
image_types = os.listdir(basedir)
cars = []
for imtype in image_types:
    cars.extend(glob.glob(basedir+imtype+'/*'))
print('Number of vehicles images found', len(cars))
with open("cars.txt", "w") as f:
    for fn in cars:
        f.write(fn+'\n')
        
# do the same for non vehicles
basedir = 'non-vehicles/'
# different folders respresent different 
image_types = os.listdir(basedir)
notcars = []
for imtype in image_types:
    notcars.extend(glob.glob(basedir+imtype+'/*'))
print('Number of non-vehicles images found', len(notcars))
with open("notcars.txt", "w") as f:
    for fn in notcars:
        f.write(fn+'\n')

Number of vehicles images found 8792
Number of non-vehicles images found 8968


In [4]:
# Define a function to compute color histogram features  
# Pass the color_space flag as 3-letter all caps string
# like 'HSV' or 'LUV' etc.
def bin_spatial(img, color_space=color_space, size=spatial_size):
    # Convert image to new color space (if specified)
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)             
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(feature_image, size).ravel() 
    # Return the feature vector
    return features

In [5]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=no_hist_bins):  #bins_range=(0, 256))
    # Compute the histogram of the RGB channels separately
    rhist = np.histogram(img[:,:,0], bins=nbins)
    ghist = np.histogram(img[:,:,1], bins=nbins)
    bhist = np.histogram(img[:,:,2], bins=nbins)
    # Generating bin centers
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    hist_features = np.ravel(hist_features) 
    return hist_features

In [6]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient = orient, pix_per_cell = pix_per_cell, cell_per_block = cell_per_block, vis=vis, feature_vector=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=spatial_size,
                                  cells_per_block=cells_per_block, transform_sqrt=False, 
                                  visualise=True, feature_vector=False)
     
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=spatial_size,
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                       visualise=False, feature_vector=False)
        return features

In [7]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
# use same order spatial, histogram, and HOG to not get messed up
def extract_features(imgs, cspace = color_space, spatial_size= spatial_size,
                        hist_bins=no_hist_bins, orient=orient, hog_channel = hog_channel,
                         pix_per_cell = pix_per_cell, cell_per_block= cell_per_block,
                           spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat ):
    # Create a list to append feature vectors to
    file_features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image) 
            
            
        if spatial_feat == True:
            # Apply bin_spatial() to get spatial color features
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist() also with a color space option now
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
            if hog_channel == "ALL":
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel],
                        orient, pix_per_cell, cell_per_block, 
                         vis = False, feature_vector = True ))
               
                
            else:
                 hog_features = get_hog_features(feature_image[:,:,hog_channel],
                            orient, pix_per_cell, cell_per_block, 
                         vis = False, feature_vector = True)
            hog_features = np.ravel(hog_features) 
            file_features.append(hog_features)       
                 
        
        # Append the new feature vector to the features list
        file_features.append(np.concatenate((spatial_features, hist_features, hog_features)))
    # Return list of feature vectors
    return file_features

In [8]:
t=time.time()
car_features = extract_features(cars, cspace=color_space, orient= orient, 
                        pix_per_cell= pix_per_cell, cell_per_block= cell_per_block, 
                        hog_channel= hog_channel)
notcar_features = extract_features(notcars, cspace=color_space, orient= orient, 
                        pix_per_cell= pix_per_cell, cell_per_block= cell_per_block, 
                        hog_channel= hog_channel)


In [17]:
def single_img_features(image, cspace=color_space, spatial_size=spatial_size,
                        hist_bins=no_hist_bins,  orient=orient, 
                        pix_per_cell= pix_per_cell, cell_per_block=cell_per_block, hog_channel = hog_channel, 
                         spatial_feat=spatial_feat, hist_feat = hist_feat, hog_feat = hog_feat): 
    
    #define a list to receive the features
    img_features = []
    # apply color conversion if other than 'RGB'
    if cspace != 'RGB':
        if cspace == 'HSV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        elif cspace == 'LUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
        elif cspace == 'HLS':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        elif cspace == 'YUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(image) 
    if spatial_feat == True:
            # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        img_features.append(spatial_features)
        print('spatial_features.shape', spatial_features.shape)
    if hist_feat == True:
            # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins)
        img_features.append(hist_features)
        print('hist_features.shape', hist_features.shape)
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vector=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vector=True)
        #8) Append features to list
        hog_features = np.ravel(hog_features) 
        print('hog_features.shape', hog_features.shape)
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

In [18]:
def visualize(figs, rows, cols, imgs,titles ):
    for i, img in enumerate(imgs):
        plt.subplt(rows, cols, i+1)
        plt.title(i+1)
        img_dims = len(img.shape)
        if img_dims < 3:
            plt.imshow(img, cmap="hot")
            plt.title(titles[i])
        else:
            plt.imshow(img)
            plt.title(titles[i])
    return

In [19]:
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
hist_bin = 16

In [20]:
car_ind = np.random.randint(0, len(cars))
notcar_ind = np.random.randint(0, len(notcars))

car_image = mpimg.imread(cars[car_ind])
notcar_image = mpimg.imread(notcars[car_ind])

In [21]:
car_features, car_hog_images = single_img_features(car_image)

spatial_features.shape (768,)
hist_features.shape (144,)
hog_features.shape (216,)


ValueError: too many values to unpack (expected 2)

In [22]:
car_features, car_hog_images = single_img_features(car_image, cspace=color_space, orient= orient, 
                        pix_per_cell= pix_per_cell, cell_per_block= cell_per_block,
                        hist_bins=no_hist_bins, 
                        spatial_feat = True, hist_feat = True, hog_feat = True,
                        spatial_size=spatial_size, hog_channel= hog_channel)


notcar_features, car_hog_images = single_img_features(notcar_image, cspace=color_space, orient= orient, 
                        pix_per_cell= pix_per_cell, cell_per_block= cell_per_block, 
                        hist_bins=no_hist_bins, 
                        spatial_feat = True, hist_feat = True, hog_feat = True,
                       
                        hog_channel= hog_channel, spatial_size=spatial_size)

spatial_features.shape (768,)
hist_features.shape (144,)
hog_features.shape (216,)


ValueError: too many values to unpack (expected 2)

In [ ]:
single_img_features(image, cspace=color_space, spatial_size=spatial_size,
                        hist_bins=no_hist_bins, hist_range=(0, 256), orient=orient, 
                        pix_per_cell=5, cell_per_block=2, hog_channel = 0, 
                         spatial_features = True, hist_features = True, hog_features = True,
                        vis = False ): 

In [ ]:
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows
    

In [ ]:


# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using spatial binning of:',spatial,
    'and', histbin,'histogram bins')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')